### Code that record servo values after moving the arm manually

In [ ]:
from Arm_Lib import Arm_Device
import time, json

arm = Arm_Device()
time.sleep(0.2)
print("DOFBOT connected")

# Optionally disable torque so you can manually or with joystick reposition
arm.Arm_serial_set_torque(0)
print("Torque off ï¿½ arm is free to move manually (joystick / by hand)")

def read_pose():
    return [arm.Arm_serial_servo_read(i) for i in range(1, 7)]

coords = []
for i in range(1, 5):
    input(f"Move arm to POINT {i}, then press ENTER ? ")
    pose = read_pose()
    print("Recorded angles:", pose)
    coords.append(pose)

with open("dofbot_pi_poses.json", "w") as f:
    json.dump(coords, f)

print("Saved poses:", coords)

# To replay ï¿½ re-enable torque, then write
arm.Arm_serial_set_torque(1)
time.sleep(0.2)

for p in coords:
    arm.Arm_serial_servo_write6(*p, 1500)  # 1.5s movement
    time.sleep(2)

print("Replay done")

### Code used to manually map every squares from a1 to h8

In [ ]:
from Arm_Lib import Arm_Device
import json, time, pathlib

arm = Arm_Device()

print("=== TEACH EVERY SQUARE BY TOUCH ===")
print("Move arm so gripper tip JUST touches the square, then press ENTER.")

board = {}         

for rank in range(1, 9):            # 1..8
    for file in range(ord('a'), ord('h')+1):   # a..h
        sq = chr(file) + str(rank)
        input(f"? {sq}  (press ENTER when touching)")
        angles = [arm.Arm_serial_servo_read(i) for i in range(1, 7)]
        board[sq] = angles + [0]      # z = 0 (touch reference) Note: was 5 before but proved to be causing issues so changed to 0
        print(f"  recorded {sq}: {angles}")

# save
pathlib.Path("/media/all_squares_touch.json").write_text(json.dumps(board, indent=2))
print("? Saved ? /media/all_squares_touch.json")

### Hard-coded E2 to E4 (To test above coordinates)

In [ ]:
import time
from Arm_Lib import Arm_Device  # or your import

arm = Arm_Device()

NEUTRAL_POS = [90,90,90,90,90,172]
E2_POS_OPEN = [84, 69, 19, 17, 264, 150]
E2_POS_CLOSE = [84, 69, 19, 17, 264, 171]
E4_POS_CLOSE = [85, 56, 19, 42, 264, 171]
E4_POS_OPEN = [85, 56, 19, 42, 264, 150]
arm.Arm_serial_servo_write6(*NEUTRAL_POS,1000)
time.sleep(1)
arm.Arm_serial_servo_write6(*E2_POS_OPEN,1000)
time.sleep(1)
arm.Arm_serial_servo_write6(*E2_POS_CLOSE,1000)
time.sleep(1)
arm.Arm_serial_servo_write6(*NEUTRAL_POS,1000)
time.sleep(1)
arm.Arm_serial_servo_write6(*E4_POS_CLOSE,1000)
time.sleep(1)
arm.Arm_serial_servo_write6(*E4_POS_OPEN,1000)
time.sleep(1)
arm.Arm_serial_servo_write6(*NEUTRAL_POS,1000)
time.sleep(1)

### Function to move any piece to any square (Workflow is same as above)

In [ ]:
import time
from Arm_Lib import Arm_Device

board = json.loads(open("/media/all_squares_touch.json").read())
arm = Arm_Device()
NEUTRAL_POS = [90,150,10,90,264,180]
NEUTRAL_POS_OPEN = [90,150,10,90,264,145]

def move(joint_list, duration=1000, wait=1.0):
    arm.Arm_serial_servo_write6(*joint_list, duration)
    time.sleep(wait)

def move_piece(src: str, dst:str):
    
    def sq_pose(sq, claw):  #Modifies servo 2 so that it hovers a little above the square instead of hitting the board
        base = board[sq][:5] + [claw]
        base[1] += 3                    
        return base

    src_close = sq_pose(src, 180)
    dst_close = sq_pose(dst, 180)
    src_open  = sq_pose(src, 140)
    dst_open  = sq_pose(dst, 140)
    
    move(NEUTRAL_POS_OPEN, 800, 1.0)
    move(src_open, 1000,1.5)
    move(src_close, 300,0.5)
    move(NEUTRAL_POS, 1000, 2.0)
    move(dst_close, 1000,1.5)
    move(dst_open, 1000,1.5)
    move(NEUTRAL_POS, 1000, 1.5)
    print(f"Moved {src} ? {dst} ?")


move_piece("e1","e2")